In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
def price_barrier_option(option_type, ticker, K, barrier_price):
    # Descargar precios históricos
    data = yf.download(ticker, start='2022-01-01', end='2025-03-15', progress=False)['Close']['SPY']
    daily_returns = data.pct_change().dropna()

    # Parámetros
    S0 = float(data.iloc[-1])
    sigma = daily_returns.std() * np.sqrt(252)
    r = 0.10
    T_days = 252
    T = 1  # años
    dt = T / T_days
    num_simulations = 10000
    Z = np.random.normal(0, 1, (num_simulations, T_days))

    returns = (r - 0.5 * sigma**2) * dt  +  sigma * np.sqrt(dt) * Z
    prices = S0 * np.cumprod(1 + returns, axis=1)

    ST = prices[:, -1]

    barrier_breached = np.any(prices >= barrier_price, axis=1)

    if option_type == 'call':

        payoff_barrier = np.where(~barrier_breached, np.maximum(ST - K, 0), 0 )

        # Payoff opcion sin barrera
        payoff_plain = np.maximum(ST - K, 0)

    elif option_type == 'put':

        payoff_barrier = np.where(~barrier_breached, np.maximum(K - ST, 0), 0 )
        # Payoff opcion sin barrera
        payoff_plain = np.maximum(K - ST, 0)

    price_barrier = np.exp(-r * T) * np.mean(payoff_barrier)
    price_plain = np.exp(-r * T) * np.mean(payoff_plain)
    price_difference = price_plain - price_barrier

    return {
        'S0': S0,
        'price_plain': price_plain,
        'price_barrier': price_barrier,
        'price_difference': price_difference
        }

In [3]:
price_barrier_option( 'put', 'SPY', 620, 630)

YF.download() has changed argument auto_adjust default to True


{'S0': 561.1220092773438,
 'price_plain': 42.66311675506483,
 'price_barrier': 34.43173436300522,
 'price_difference': 8.231382392059608}